In [1]:
import csv
import numpy as np
import pandas as pd
from scipy import stats
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import random
import numpy as np
import json

In [2]:
from scipy.cluster.vq import vq, kmeans, whiten

# Loading data

In [3]:
#path to stimuli (cocoids)
all_stimulus_path = ''

#path to clip embeddings, ordered by all_stimulus_path
clip_embs_path = ''

#path to resnet embeddings, ordered by all_stimulus_path
resnet_embs_path = ''

#exact coco ids used for clustering 
stim_ids_path = '/data/non_shared_food_coco_ids.npy'

#base path to voxel hcp mask
voxel_hcp_mask_path = ''

#base path to food v all mask
voxel_food_v_all_mask_path = ''

#base preprocessed voxels path 
voxels_path = ''

In [4]:
#voxels embedding

subj=1
path = voxel_hcp_mask_path.format(subj)
voxels = np.load(path)
indices = voxels.nonzero()

path2 = voxel_food_v_all_mask_path.format(subj)
voxels2 = np.load(path2)
indices2 = np.where(voxels2<=0.05)

indices0_set = set(indices[0].tolist())
indices2_set = set(indices2[0].tolist())
intersec_indices = list(indices0_set.intersection(indices2_set))

voxels = np.load(voxels_path.format(subj))[:, intersec_indices]

In [ ]:
clip_embs = np.load(clip_embs_path)
resnet_embs = np.load(resnet_embs_path)
voxel_embs = voxels

#choose which embedding
embs = clip_embs
all_stimulus = np.load(all_stimulus_path)
stim_ids = np.load(stim_ids_path)

# COCO loading

In [ ]:
dataDir = ''
dataType = 'train2017'
dataType_val = 'val2017'
annFile = '{}/instances_{}.json'.format(dataDir, dataType)
annFile2 = '{}/instances_{}.json'.format(dataDir, dataType_val)

In [ ]:
coco = COCO(annFile)
coco2 = COCO(annFile2)

In [ ]:
def displayCOCOImg(coco_ids, cluster=-1, download=False, output_dir = ''):
    count = 0
    for coco_id in coco_ids:
        print("coco_ID: ", coco_id)
        try:
            img = coco.loadImgs(coco_id)
            img = img[0]
            I = io.imread(img['coco_url'])
            
            #download
            if download:
                tarDir = output_dir
                fname = tarDir  + img['file_name']
                urlretrieve(img['coco_url'], fname)
            
            plt.imshow(I); 
            plt.axis('off')
            plt.show()
        except:
            img = coco2.loadImgs(coco_id)
            img = img[0]
            I = io.imread(img['coco_url'])
            
            #download
            if download:
                tarDir = output_dir
                fname = tarDir  + img['file_name']
                urlretrieve(img['coco_url'], fname)
            
            plt.imshow(I); 
            plt.axis('off')
            plt.show()
        count += 1

# Cluster

In [ ]:
embs_subset = embs[stim_ids]
stimulus_subset = all_stimulus[stim_ids]

In [ ]:
def getClusters(embs_subset, num_clusters):
    whitened = whiten(embs_subset)
    book = np.array((whitened[0],whitened[2]))
    centroids = kmeans(whitened.astype(float),num_clusters)
    
    assignments = []
    for i in range(len(embs_subset)):
        dists = []
        for centroid_i in range(len(centroids[0])):
            curr_dist = np.linalg.norm(embs_subset[i] - centroids[0][centroid_i])
            dists.append(curr_dist)
        dists_np = np.asarray(dists)
        min_centroid = np.argmin(dists_np)
        
        assignments.append(min_centroid)
    
    all_inds = []
    all_stimuli_subset = []
    for centroid_i in range(num_clusters):
        curr_centroid_inds = np.where(np.asarray(assignments) == centroid_i)[0]
        all_inds.append(curr_centroid_inds)
        
    return centroids, all_inds

In [ ]:
centroids, centroid_ids = getClusters(embs_subset, 4)

In [ ]:
def getTopImgsInCluster(cluster_i, top_n=10):
    inds = np.asarray(centroid_ids[cluster_i])
    
    centroids_tiled = np.tile(centroids[0][cluster_i], (len(inds), 1))
    sums = np.sqrt(np.sum(np.square(embs_subset[inds] - centroids_tiled), 1))
    top_stimuli = stimulus_subset[inds][(sums).argsort()[:top_n]]
    
    for stim in top_stimuli:
        displayCOCOImg([int(stim)], cluster_i+1)

In [ ]:
num_data = 0
for i in range(len(centroid_ids)):
    num_data += len(centroid_ids[i])
num_data

same_cluster_mtx = np.zeros((num_data,num_data))
for cluster_num in range(4):
    for i in range(len(centroid_ids[cluster_num])):
        for j in range(i, len(centroid_ids[cluster_num])):
            same_cluster_mtx[centroid_ids[cluster_num][i]][centroid_ids[cluster_num][j]] = 1
            same_cluster_mtx[centroid_ids[cluster_num][j]][centroid_ids[cluster_num][i]] = 1 